In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os

#Adding datatime dependency to be able to get the current date each time code is run
import datetime

# Import API key
# from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

#Import pprint
from pprint import pprint

# Google developer API key
from config import gkey

# import gmaps
import gmaps

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
#define path to data containing the address info for each school and read in to a dataframe
path_to_file = os.path.join(".." , "rawdata" ,"SchoolDirectory2.csv")

school_desc_df = pd.read_csv(path_to_file)

#just keep the fields that will be used
school_desc_df=school_desc_df[['School Name','School Number', 'District Type', 'School Site Street Address', 'School Site City', 'School Site State', 'School Site Zip', 'Grade Range']]
school_desc_df.head()

#the school number was stored with an apostrophe in front (ie. '001902001), so removing the apostrophe and changing to numeric
#to get rid of any zeroes stored in front of number
school_desc_df['School Number 2']=school_desc_df['School Number'].str[1:]
school_desc_df['School Number 2']=school_desc_df['School Number 2'].apply(pd.to_numeric)
school_desc_df.head()

#Create a list that contains the school number  and a full address that Combines the street address, 
# school, and zip code into a single field that can be passed to Google geocode
fulladdress = []
school_desc_df[['School Number', 'School Name', 'School Site Street Address', 'School Site City', 'School Site Zip']].head()

# fulladdress=[school_desc_df['School Number 2'], [school_desc_df['School Site Street Address'] + ' ' + school_desc_df['School Site City'] + ',TX ' + school_desc_df['School Site Zip']]]
fulladdress=school_desc_df['School Site Street Address'] + ' ' + school_desc_df['School Site City'] + ',TX ' + school_desc_df['School Site Zip']
schoolnum=school_desc_df['School Number 2']

fulladdress=fulladdress.tolist()
schoolnum=schoolnum.tolist()
type(fulladdress)

list

In [3]:

# create lists to hold latitudes and longitudes
lats = []
lngs = []

#for testing, just use a small subset of the addresses 

for address in fulladdress[0:5]:
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(address, gkey)
    
    response = requests.get(target_url)
 
    # Convert the response to json
    response_json = response.json()

    # store lat/long data in variables that are then added to alist
    lat = response_json["results"][0]["geometry"]["location"]["lat"]
    lats.append(lat)
    
    lng = response_json["results"][0]["geometry"]["location"]["lng"]
    lngs.append(lng)





In [4]:
fig=gmaps.figure()

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Combine list of lats and lngs into list of tuples
coordinates=tuple(zip(lats,lngs))

print(coordinates)

# Create a marker layer using our coordinates
markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
fig.add_layer(markers)

# display the figure with the newly added layer
fig

((31.9229638, -95.92387049999999), (31.9229638, -95.92387049999999), (31.9229638, -95.92387049999999), (31.6558007, -95.58306929999999), (31.7731815, -95.6398186))


Figure(layout=FigureLayout(height='420px'))

In [96]:
# create a data frame with schoolnum, full address at latitude and longitue
# use a subset of the data to start with to test code
schoolnumshort=schoolnum[0:5]
fulladdressshort=fulladdress[0:5]

location_df=pd.DataFrame(np.column_stack([schoolnumshort, fulladdressshort, lats, lngs]), 
                        columns=['schoolnun', 'fulladdress', 'latitude', 'longitude'])

location_df.head()



,schoolnun,fulladdress,latitude,longitude
0,1902001,"17750 N US HWY 287 TENNESSEE COLONY,TX 75861",31.9229638,-95.92387049999999
1,1902041,"17750 N US HWY 287 TENNESSEE COLONY,TX 75861",31.9229638,-95.92387049999999
2,1902103,"17750 N US HWY 287 TENNESSEE COLONY,TX 75861",31.9229638,-95.92387049999999
3,1903001,"9002 S STATE HWY 19 ELKHART,TX 75839-9701",31.6558007,-95.58306929999999
4,1903002,"420 W PALESTINE AVE PALESTINE,TX 75801",31.7731815,-95.6398186


In [102]:
#add columns to hold the results of a search for nearby ice cream shops and nearby universities

location_df['icecream']=0
location_df['university']=0

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in location_df.iterrows():
    location=location_df.loc[index, 'latitude'] + ',' + location_df.loc[index,'longitude']
    
    # Search to see if there are any places that serve ice cream near each school    
    target_radius=1000
    type1='restaurant'
    keyword1='ice cream'
    
    params1 = {
    "location": location,
    "type": type1,
    "radius" : target_radius,
    "keyword" : keyword1,
    "key": gkey,
    }
        
    response = requests.get(base_url, params=params1)

    response_json=response.json()
    icecreamstatus=response_json['status']

    if icecreamstatus == "OK":
        location_df.loc[index, 'icecream'] = 1
        
    # Search to see if there is a university near each school    
    target_radius2=5000
    type2='university'
    
    params1 = {
    "location": location,
    "type": type2,
    "radius" : target_radius2,
    "key": gkey,
    }
        
    response = requests.get(base_url, params=params1)

    response_json=response.json()
    univstatus=response_json['status']

    if univstatus == "OK":
        location_df.loc[index, 'university'] = 1    
        
    
    
    
#     # extract results
#     results = response['results']
    
#     # Use exception handling in case any of our restaurants' attributes are missing
#     # NOTE: if our code hits a KeyError or IndexError on ANY of the lines of code below, it will skip any unexecuted code and move to the next iteration
#     try:
#         print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
#         types_df.loc[index, 'name'] = results[0]['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
#         types_df.loc[index, 'price_level'] = results[0]['price_level']
#         types_df.loc[index, 'rating'] = results[0]['rating']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")


pprint(response_json)




{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 31.7465031, 'lng': -95.6174582},
                           'viewport': {'northeast': {'lat': 31.7477105302915,
                                                      'lng': -95.6163372197085},
                                        'southwest': {'lat': 31.7450125697085,
                                                      'lng': -95.6190351802915}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png',
              'id': 'bb12a091882d3a55c6f872d27fe07aff015a3ed0',
              'name': 'Christian Heritage Academy',
              'place_id': 'ChIJ2wwdYkRkSIYRvWBUnbONfZI',
              'plus_code': {'compound_code': 'P9WM+J2 Palestine, Texas, United '
                                             'States',
                            'global_code': '8636P9WM+J2'},
              'rating': 4.2,
              'reference': 'ChIJ2wwdYkRkSIYRvWBUnbONfZI',
              'scope': 'GOOG

In [103]:
location_df.head()

,schoolnun,fulladdress,latitude,longitude,icecream,university
0,1902001,"17750 N US HWY 287 TENNESSEE COLONY,TX 75861",31.9229638,-95.92387049999999,0,0
1,1902041,"17750 N US HWY 287 TENNESSEE COLONY,TX 75861",31.9229638,-95.92387049999999,0,0
2,1902103,"17750 N US HWY 287 TENNESSEE COLONY,TX 75861",31.9229638,-95.92387049999999,0,0
3,1903001,"9002 S STATE HWY 19 ELKHART,TX 75839-9701",31.6558007,-95.58306929999999,0,1
4,1903002,"420 W PALESTINE AVE PALESTINE,TX 75801",31.7731815,-95.6398186,1,1
